In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 8.0 MB/s eta 0:00:00


In [ ]:
import torch
import optuna
import random
import pandas as pd
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [ ]:
class TransformerBlock(nn.Module): #build a transformer block with dropout and normilization
    def __init__(self, input_dim, num_heads, ffn_dim, dropout):
        super(TransformerBlock, self).__init__()
        self.attention = nn.MultiheadAttention(embed_dim=input_dim, num_heads=num_heads, dropout=dropout) #multi-head self-attention layer
        self.norm1 = nn.LayerNorm(input_dim) #first normalization after attention
        self.norm2 = nn.LayerNorm(input_dim) #second normalization after FFN
        self.ffn = nn.Sequential( #feed-forward network with ReLU activation
            nn.Linear(input_dim, ffn_dim),
            nn.ReLU(),
            nn.Linear(ffn_dim, input_dim)
        )
        self.dropout = nn.Dropout(dropout) #dropout for regularization

    def forward(self, x):
        attn_output, _ = self.attention(x, x, x) #self-attention: query, key, value are the same
        x = self.norm1(x + self.dropout(attn_output)) #dropout + norm after attention

        ffn_output = self.ffn(x) #pass through feed-forward network
        x = self.norm2(x + self.dropout(ffn_output)) #dropout + norm after FFN

        return x

class TransformerMLP(nn.Module): #transformer MLP for numerical data
    def __init__(self, input_dim=62, hidden_dim=128, num_heads=4, num_layers=2, ffn_dim=128, dropout=0.1, output_dim=8):
        super(TransformerMLP, self).__init__()

        assert hidden_dim % num_heads == 0, "hidden_dim must be divisible by num_heads" #assertion for optuna optimization, otherwise model will error

        #standard transformer with embedding layer, attention heads, and output layer
        self.embedding = nn.Linear(input_dim, hidden_dim) #linear layer to project input into hidden_dim
        self.transformer_layers = nn.ModuleList([ #stack of transformer blocks
            TransformerBlock(hidden_dim, num_heads, ffn_dim, dropout) for _ in range(num_layers)
        ])
        self.fc = nn.Linear(hidden_dim, output_dim) #final linear layer for output

    def forward(self, x):
        x = self.embedding(x) #project input to hidden space
        x = x.unsqueeze(1) #add sequence dimension (seq_len=1 for tabular input)

        for layer in self.transformer_layers:
            x = layer(x) #pass through each transformer block

        x = x.squeeze(1) #remove sequence dimension
        x = self.fc(x) #map to output dimension
        return x

In [ ]:
#standard train loop
def train(model, data_loader, crit, opt, device): #model, trainset data loader, criterion, optimizer, device
  model.train()
  total_loss, correct, total = 0, 0, 0
  total_size = len(data_loader)
  for data, label in data_loader:
    data= data.to(device)
    label = label.to(device)

    opt.zero_grad()

    out = model(data)
    loss = crit(out, label)
    loss.backward()
    opt.step()

    total_loss += loss.item()
    _, y = torch.max(out, 1)
    total += label.size(0)
    correct += (y == label).sum().item()

  epoch_loss = total_loss / total_size
  epoch_score = correct/total

  return epoch_loss, epoch_score

In [ ]:
#standard eval loop
def eval(model, data_loader, crit, device): #model, testset data loader, criterion, device
  model.eval()
  total_loss, correct, total = 0, 0, 0
  total_size = len(data_loader)
  with torch.no_grad():
    for data, label in data_loader:
      data= data.to(device)
      label = label.to(device)

      out = model(data)
      loss = crit(out, label)

      total_loss += loss.item()
      _, y = torch.max(out, 1)
      total += label.size(0)
      correct += (y == label).sum().item()

  epoch_loss = total_loss / total_size
  epoch_score = correct/total
  return epoch_loss, epoch_score

In [ ]:
def run_model(model, train_loader, test_loader, epochs, crit, opt, device):
  final_test_score = 0
  for epoch in range(epochs):
    train_loss, train_score = train(model, train_loader, crit, opt, device)
    test_loss, test_score = eval(model, test_loader, crit, device)

    print(f'Epoch {epoch+1}/{epochs}')
    print(f'Train Loss: {train_loss:.4f}, Train Score: {train_score:.4f}')
    print(f'Test Loss: {test_loss:.4f}, Test Score: {test_score:.4f}')
    final_test_score = test_score
  return final_test_score

In [ ]:
#Custom pandas dataset given csvs are read in as dataframes
class PandasDataset(Dataset):
    def __init__(self, dataframe, target_column=None):
        self.features = dataframe.drop(columns=[target_column]).values if target_column else dataframe.values
        self.features = torch.tensor(self.features, dtype=torch.float)

        if target_column:
            self.labels = torch.tensor(dataframe[target_column].values, dtype=torch.int64)
        else:
            self.labels = None

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        if self.labels is not None:
            return self.features[idx], self.labels[idx]
        return self.features[idx]

In [ ]:
label_data = pd.read_csv('test_dataset_label_encoded.csv', delimiter='\t') #read in label encoded data
label_data['Placement'] = label_data['Placement'] -1 #get placement aka independant variable
label_data_traitless = label_data.copy(deep=True).iloc[:,:55] #make a subset without trait data
label_data_traitless["Placement"] = label_data['Placement']
match_data = pd.read_csv('test_dataset_encoded.csv', delimiter='\t') #repeat steps for binary encoded data
match_data['Placement'] = match_data['Placement'] -1
match_data_traitless = label_data.copy(deep=True).iloc[:,:344]
match_data_traitless["Placement"] = match_data['Placement']
match_data

,Champion 1_0,Champion 1_1,Champion 1_2,Champion 1_3,Champion 1_4,Champion 1_5,Champion 1_6,Level 1,Item 1 1_0,Item 1 1_1,...,Trait 7_3,Trait 7_4,Tier 7,Trait 8_0,Trait 8_1,Trait 8_2,Trait 8_3,Trait 8_4,Tier 8,Placement
0,0,0,0,0,0,0,1,2,0,0,...,0,1,3,0,0,0,0,1,0,1
1,0,0,0,0,0,1,0,2,0,0,...,1,0,0,0,0,0,0,1,0,2
2,0,0,0,0,0,1,1,2,0,0,...,1,0,0,0,0,0,0,1,0,6
3,0,0,0,0,1,0,0,2,0,0,...,1,0,0,0,0,0,0,1,0,4
4,0,0,0,0,0,1,0,2,0,0,...,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270891,0,0,0,0,0,0,1,2,0,0,...,0,1,3,0,0,0,0,1,0,2
270892,0,0,0,0,1,0,0,3,1,0,...,1,0,0,0,0,0,0,1,0,5
270893,0,0,0,0,1,0,1,2,0,0,...,1,0,0,0,0,0,0,1,0,6
270894,0,0,0,0,0,0,1,2,0,0,...,1,0,0,0,0,0,0,1,0,4


**Training on the 4 different datasets to see what performs the best before optimizing it**

In [ ]:
train_binary, test_binary = train_test_split(match_data, test_size=0.2, random_state=42, shuffle=True)
train_binary_traitless, test_binary_traitless = train_test_split(match_data_traitless, test_size=0.2, random_state=42, shuffle=True)
train_label, test_label = train_test_split(label_data, test_size=0.2, random_state=42, shuffle=True)
train_label_traitless, test_label_traitless = train_test_split(label_data_traitless, test_size=0.2, random_state=42, shuffle=True)

In [ ]:
train_dataset_binary = PandasDataset(train_binary, target_column="Placement")
test_dataset_binary = PandasDataset(test_binary, target_column="Placement")

    # Create DataLoaders
train_loader_binary = DataLoader(train_dataset_binary, batch_size=1024, shuffle=True)
test_loader_binary = DataLoader(test_dataset_binary, batch_size=1024, shuffle=False)

In [ ]:
train_dataset_binary_traitless = PandasDataset(train_binary_traitless, target_column="Placement")
test_dataset_binary_traitless = PandasDataset(test_binary_traitless, target_column="Placement")

    # Create DataLoaders
train_loader_binary_traitless = DataLoader(train_dataset_binary_traitless, batch_size=1024, shuffle=True)
test_loader_binary_traitless = DataLoader(test_dataset_binary_traitless, batch_size=1024, shuffle=False)

In [ ]:
train_dataset_label = PandasDataset(train_label, target_column="Placement")
test_dataset_label = PandasDataset(test_label, target_column="Placement")

    # Create DataLoaders
train_loader_label = DataLoader(train_dataset_label, batch_size=1024, shuffle=True)
test_loader_label = DataLoader(test_dataset_label, batch_size=1024, shuffle=False)

In [ ]:
train_dataset_label_traitless = PandasDataset(train_label_traitless, target_column="Placement")
test_dataset_label_traitless = PandasDataset(test_label_traitless, target_column="Placement")

    # Create DataLoaders
train_loader_label_traitless = DataLoader(train_dataset_label_traitless, batch_size=1024, shuffle=True)
test_loader_label_traitless = DataLoader(test_dataset_label_traitless, batch_size=1024, shuffle=False)

In [ ]:
train_dataset_binary[0]

(tensor([0., 0., 0., 0., 0., 1., 0., 2., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 1., 0., 3., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.,
         1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 3.,
         0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0.,
         0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 1., 1., 0., 2., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
         0., 1., 1., 2., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1.,
         0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0.,
         0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 1., 0.,

In [ ]:
epochs=15
crit = nn.CrossEntropyLoss()
learning_rate = 0.001

In [ ]:
binary_model = TransformerMLP(input_dim=len(train_dataset_binary[0][0]), hidden_dim=512).to(device)
opt = torch.optim.Adam(binary_model.parameters(), lr=learning_rate)
final_test_score = run_model(binary_model, train_loader_binary, test_loader_binary, epochs, crit, opt, device)

Epoch 1/15
Train Loss: 1.8801, Train Score: 0.2381
Test Loss: 1.7342, Test Score: 0.2831
Epoch 2/15
Train Loss: 1.6848, Train Score: 0.2951
Test Loss: 1.6657, Test Score: 0.3023
Epoch 3/15
Train Loss: 1.6586, Train Score: 0.3047
Test Loss: 1.6523, Test Score: 0.3119
Epoch 4/15
Train Loss: 1.6360, Train Score: 0.3110
Test Loss: 1.6368, Test Score: 0.3148
Epoch 5/15
Train Loss: 1.6199, Train Score: 0.3183
Test Loss: 1.6404, Test Score: 0.3086
Epoch 6/15
Train Loss: 1.6145, Train Score: 0.3191
Test Loss: 1.6149, Test Score: 0.3190
Epoch 7/15
Train Loss: 1.5974, Train Score: 0.3265
Test Loss: 1.6122, Test Score: 0.3213
Epoch 8/15
Train Loss: 1.5907, Train Score: 0.3293
Test Loss: 1.6174, Test Score: 0.3210
Epoch 9/15
Train Loss: 1.5854, Train Score: 0.3298
Test Loss: 1.6138, Test Score: 0.3181
Epoch 10/15
Train Loss: 1.5796, Train Score: 0.3330
Test Loss: 1.6087, Test Score: 0.3201
Epoch 11/15
Train Loss: 1.5872, Train Score: 0.3312
Test Loss: 1.6239, Test Score: 0.3180
Epoch 12/15
Train L

In [ ]:
binary_model_traitless = TransformerMLP(input_dim=len(train_dataset_binary_traitless[0][0]), hidden_dim=512).to(device)
opt = torch.optim.Adam(binary_model_traitless.parameters(), lr=learning_rate)
final_test_score_traitless = run_model(binary_model_traitless, train_loader_binary_traitless, test_loader_binary_traitless, epochs, crit, opt, device)

Epoch 1/15
Train Loss: 1.9341, Train Score: 0.2194
Test Loss: 1.8567, Test Score: 0.2446
Epoch 2/15
Train Loss: 1.8443, Train Score: 0.2441
Test Loss: 1.8426, Test Score: 0.2463
Epoch 3/15
Train Loss: 1.8777, Train Score: 0.2324
Test Loss: 1.9842, Test Score: 0.1802
Epoch 4/15
Train Loss: 1.9720, Train Score: 0.2015
Test Loss: 1.9279, Test Score: 0.2121
Epoch 5/15
Train Loss: 1.9544, Train Score: 0.2090
Test Loss: 1.9307, Test Score: 0.2199
Epoch 6/15
Train Loss: 1.9324, Train Score: 0.2167
Test Loss: 1.8822, Test Score: 0.2291
Epoch 7/15
Train Loss: 1.9097, Train Score: 0.2240
Test Loss: 1.9073, Test Score: 0.2243
Epoch 8/15
Train Loss: 1.9554, Train Score: 0.2085
Test Loss: 1.9050, Test Score: 0.2287
Epoch 9/15
Train Loss: 1.9011, Train Score: 0.2253
Test Loss: 1.9392, Test Score: 0.2098
Epoch 10/15
Train Loss: 1.8898, Train Score: 0.2309
Test Loss: 1.8563, Test Score: 0.2391
Epoch 11/15
Train Loss: 1.8601, Train Score: 0.2400
Test Loss: 1.8426, Test Score: 0.2449
Epoch 12/15
Train L

In [ ]:
label_model = TransformerMLP(input_dim=len(train_dataset_label[0][0]), hidden_dim=512).to(device)
opt = torch.optim.Adam(label_model.parameters(), lr=learning_rate)
final_test_score_label = run_model(label_model, train_loader_label, test_loader_label, epochs, crit, opt, device)

Epoch 1/15
Train Loss: 1.9440, Train Score: 0.2173
Test Loss: 1.8468, Test Score: 0.2429
Epoch 2/15
Train Loss: 1.8311, Train Score: 0.2476
Test Loss: 1.8195, Test Score: 0.2493
Epoch 3/15
Train Loss: 1.8163, Train Score: 0.2538
Test Loss: 1.8092, Test Score: 0.2596
Epoch 4/15
Train Loss: 1.8149, Train Score: 0.2534
Test Loss: 1.8028, Test Score: 0.2572
Epoch 5/15
Train Loss: 1.8620, Train Score: 0.2394
Test Loss: 1.9554, Test Score: 0.2065
Epoch 6/15
Train Loss: 1.9990, Train Score: 0.1862
Test Loss: 2.0063, Test Score: 0.1797
Epoch 7/15
Train Loss: 2.0584, Train Score: 0.1457
Test Loss: 2.0797, Test Score: 0.1379
Epoch 8/15
Train Loss: 2.0426, Train Score: 0.1632
Test Loss: 2.0241, Test Score: 0.1807
Epoch 9/15
Train Loss: 2.0237, Train Score: 0.1722
Test Loss: 2.0892, Test Score: 0.1413
Epoch 10/15
Train Loss: 1.9654, Train Score: 0.2041
Test Loss: 2.0433, Test Score: 0.1614
Epoch 11/15
Train Loss: 1.9383, Train Score: 0.2156
Test Loss: 1.9249, Test Score: 0.2221
Epoch 12/15
Train L

In [ ]:
label_model_traitless = TransformerMLP(input_dim=len(train_dataset_label_traitless[0][0]), hidden_dim=512).to(device)
opt = torch.optim.Adam(label_model_traitless.parameters(), lr=learning_rate)
final_test_score_label_traitless = run_model(label_model_traitless, train_loader_label_traitless, test_loader_label_traitless, epochs, crit, opt, device)

Epoch 1/15
Train Loss: 1.9403, Train Score: 0.2190
Test Loss: 1.8428, Test Score: 0.2464
Epoch 2/15
Train Loss: 1.8520, Train Score: 0.2415
Test Loss: 1.8315, Test Score: 0.2480
Epoch 3/15
Train Loss: 1.8526, Train Score: 0.2418
Test Loss: 1.8355, Test Score: 0.2480
Epoch 4/15
Train Loss: 1.8324, Train Score: 0.2484
Test Loss: 1.8200, Test Score: 0.2499
Epoch 5/15
Train Loss: 1.8145, Train Score: 0.2541
Test Loss: 1.8009, Test Score: 0.2581
Epoch 6/15
Train Loss: 1.8079, Train Score: 0.2573
Test Loss: 1.8128, Test Score: 0.2554
Epoch 7/15
Train Loss: 1.8073, Train Score: 0.2570
Test Loss: 1.8007, Test Score: 0.2564
Epoch 8/15
Train Loss: 1.9857, Train Score: 0.1946
Test Loss: 2.0613, Test Score: 0.1407
Epoch 9/15
Train Loss: 1.9897, Train Score: 0.1934
Test Loss: 1.9575, Test Score: 0.2121
Epoch 10/15
Train Loss: 2.0335, Train Score: 0.1600
Test Loss: 2.0781, Test Score: 0.1318
Epoch 11/15
Train Loss: 2.0441, Train Score: 0.1649
Test Loss: 1.9496, Test Score: 0.2080
Epoch 12/15
Train L

**binary encoded data with traits performs the best, so optimize on this model**

In [ ]:
def objective(trial):
  num_heads = trial.suggest_int("num_heads", 2, 8)
  random_integer = random.randint(32, 128)
  hidden_dim = (random_integer // 32) * 32 * num_heads
  num_layers = trial.suggest_int("num_layers", 1, 4)
  ffn_dim = trial.suggest_int("ffn_dim", 64, 512, log=True)
  dropout = trial.suggest_uniform("dropout", 0.1, 0.5)
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-1)
  batch_size = trial.suggest_int("batch_size", 16, 1024)

  input_dim=len(train_dataset_binary[0][0])
  best_model = TransformerMLP(input_dim=input_dim,
                              hidden_dim=hidden_dim,
                              num_heads=num_heads,
                              num_layers=num_layers,
                              ffn_dim=ffn_dim,
                              dropout=dropout,
                              output_dim=8)
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(best_model.parameters(), lr=learning_rate)

  train_loader_binary_best = DataLoader(train_dataset_binary, batch_size=batch_size, shuffle=True)

  best_model.train()
  running_loss = 0.0
  correct_preds = 0
  total_preds = 0
  print(num_heads, hidden_dim, num_layers, ffn_dim, dropout, learning_rate, batch_size)
  for inputs, labels in train_loader_binary_best:
      optimizer.zero_grad()

      # Forward pass
      outputs = best_model(inputs)
      loss = criterion(outputs, labels)

      # Backpropagation
      loss.backward()
      optimizer.step()

      running_loss += loss.item()

      # Calculate accuracy
      _, predicted = torch.max(outputs.data, 1)
      total_preds += labels.size(0)
      correct_preds += (predicted == labels).sum().item()

  # Calculate accuracy
  accuracy = correct_preds / total_preds
  print(f"Loss: {running_loss / len(train_loader_binary_best)}, Accuracy: {accuracy}")

  # Return the loss as the value to optimize
  return running_loss / len(train_loader_binary_best)

# Create an Optuna study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

# Output the best hyperparameters found by Optuna
print("Best hyperparameters found: ", study.best_trial.params)

[I 2025-03-12 19:00:06,120] A new study created in memory with name: no-name-582be93f-1f70-4eec-8cb2-2117980fb6f0
<ipython-input-35-cd7310807b32>:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout = trial.suggest_uniform("dropout", 0.1, 0.5)
<ipython-input-35-cd7310807b32>:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-1)


3 192 4 111 0.14047986072797786 0.017246153310800715 672


[I 2025-03-12 19:03:20,842] Trial 0 finished with value: 2.124136547555126 and parameters: {'num_heads': 3, 'num_layers': 4, 'ffn_dim': 111, 'dropout': 0.14047986072797786, 'learning_rate': 0.017246153310800715, 'batch_size': 672}. Best is trial 0 with value: 2.124136547555126.


Loss: 2.124136547555126, Accuracy: 0.12582827294708282
5 480 1 258 0.18821946248479252 0.0005203017550857601 259


[I 2025-03-12 19:03:58,000] Trial 1 finished with value: 1.7089815598377334 and parameters: {'num_heads': 5, 'num_layers': 1, 'ffn_dim': 258, 'dropout': 0.18821946248479252, 'learning_rate': 0.0005203017550857601, 'batch_size': 259}. Best is trial 1 with value: 1.7089815598377334.


...

Loss: 1.7308268034888679, Accuracy: 0.28114213994352055
3 192 3 230 0.16697663162953597 0.0005998757494016452 48


[I 2025-03-12 20:19:12,363] Trial 48 finished with value: 1.6932455801620567 and parameters: {'num_heads': 3, 'num_layers': 3, 'ffn_dim': 230, 'dropout': 0.16697663162953597, 'learning_rate': 0.0005998757494016452, 'batch_size': 48}. Best is trial 22 with value: 1.6853186340378665.


Loss: 1.6932455801620567, Accuracy: 0.29158899204488825
6 384 3 229 0.3543911100502931 0.00035628653913875055 125


[I 2025-03-12 20:20:33,981] Trial 49 finished with value: 1.7068649351665581 and parameters: {'num_heads': 6, 'num_layers': 3, 'ffn_dim': 229, 'dropout': 0.3543911100502931, 'learning_rate': 0.00035628653913875055, 'batch_size': 125}. Best is trial 22 with value: 1.6853186340378665.


Loss: 1.7068649351665581, Accuracy: 0.2884420162793702
Best hyperparameters found:  {'num_heads': 2, 'num_layers': 2, 'ffn_dim': 267, 'dropout': 0.10042809904090386, 'learning_rate': 0.0011073882093840127, 'batch_size': 106}


**Now build model given optimal hyperparameters according to optuna**

In [ ]:
binary_model_best = TransformerMLP(input_dim=len(train_dataset_binary[0][0]), hidden_dim=64, num_heads=2, num_layers=2, ffn_dim=267, dropout=.1004, output_dim=8).to(device)
opt = torch.optim.Adam(binary_model_best.parameters(), lr=0.0011079)
final_test_score = run_model(binary_model_best, train_loader_binary, test_loader_binary, epochs, crit, opt, device)

Epoch 1/15
Train Loss: 1.7485, Train Score: 0.2760
Test Loss: 1.6642, Test Score: 0.3057
Epoch 2/15
Train Loss: 1.6443, Train Score: 0.3091
Test Loss: 1.6341, Test Score: 0.3166
Epoch 3/15
Train Loss: 1.6204, Train Score: 0.3188
Test Loss: 1.6261, Test Score: 0.3170
Epoch 4/15
Train Loss: 1.6024, Train Score: 0.3252
Test Loss: 1.6394, Test Score: 0.3110
Epoch 5/15
Train Loss: 1.5898, Train Score: 0.3299
Test Loss: 1.6068, Test Score: 0.3223
Epoch 6/15
Train Loss: 1.5806, Train Score: 0.3329
Test Loss: 1.6084, Test Score: 0.3221
Epoch 7/15
Train Loss: 1.5711, Train Score: 0.3376
Test Loss: 1.5994, Test Score: 0.3243
Epoch 8/15
Train Loss: 1.5619, Train Score: 0.3420
Test Loss: 1.6018, Test Score: 0.3240
Epoch 9/15
Train Loss: 1.5524, Train Score: 0.3453
Test Loss: 1.6044, Test Score: 0.3219
Epoch 10/15
Train Loss: 1.5459, Train Score: 0.3478
Test Loss: 1.6044, Test Score: 0.3203
Epoch 11/15
Train Loss: 1.5375, Train Score: 0.3528
Test Loss: 1.6007, Test Score: 0.3237
Epoch 12/15
Train L